In [8]:
import ccxt
import time
import datetime
from datetime import datetime as dt
from zoneinfo import ZoneInfo
import concurrent.futures

# Replace with your actual API keys for Bybit, Bitget, and OKX
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
OKX_API_KEY = '48af790d-3250-44ec-9528-54d5cd1c1009'
OKX_API_SECRET = 'C91F9F5826FD2F5A8ADC91D945643AD9'
OKX_API_PASSWORD = 'OkX!?Secure789Trade2025?!'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {
        'adjustForTimeDifference': True,
        'recvWindow': 10000
    }
})

# Connect to Bitget API
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to OKX API
okx = ccxt.okx({
    'apiKey': OKX_API_KEY,
    'secret': OKX_API_SECRET,
    'password': OKX_API_PASSWORD,
    'options': {'adjustForTimeDifference': True}
})



In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.1 / 100  # 0.1% for testing
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds

# Adjustable trading fees per exchange
TRADING_FEES = {
    'Bybit': 0.001,   # 0.1%
    'Bitget': 0.0008, # 0.08%
    'OKX': 0.0007     # 0.07%
}

exchanges = {'Bybit': bybit, 'Bitget': bitget, 'OKX': okx}

# Cache futures markets
futures_markets_cache = {}

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol]

        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if exchange_name == "OKX" and market.get('type') != 'spot':
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        ticker = exchange.fetch_ticker(symbol)
        return ticker.get("last", None)
    except Exception:
        return None

def fetch_prices_concurrently(exchange, symbols):
    prices = {}
    def fetch(symbol):
        prices[symbol] = get_price(exchange, symbol)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(fetch, symbols)
    return prices

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_prices, futures_prices):
    spot_price = spot_prices.get(symbol)
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbol = find_futures_symbol(symbol, futures_list)

    if not futures_symbol:
        return

    futures_price = futures_prices.get(futures_symbol)
    if not spot_price or not futures_price:
        return

    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)

    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(ZoneInfo("Asia/Jakarta")).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print(f"    ⚡ Execute arbitrage on {exchange_name}")
        print("-" * 60)

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbols = [find_futures_symbol(t, futures_list) for t in tokens if find_futures_symbol(t, futures_list)]

    spot_prices = fetch_prices_concurrently(exchange, tokens)
    futures_prices = fetch_prices_concurrently(exchange, futures_symbols)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_intra_exchange_arbitrage, exchange, exchange_name, token, spot_prices, futures_prices) for token in tokens]
        concurrent.futures.wait(futures)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()


✅ Found 402 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
🔍 Arbitrage Opportunity on Bybit: MYRIA/USDT | 2025-04-02 00:24:43 WIB
    ✅ Spot Price: 0.00079 USDT
    ✅ Futures Price: 0.00080 USDT
    ✅ Spread: 0.3531%
    ✅ Net Profit After Fees: 0.1531%
    ⚡ Execute arbitrage on Bybit
------------------------------------------------------------
🔍 Arbitrage Opportunity on Bybit: OM/USDT | 2025-04-02 00:24:43 WIB
    ✅ Spot Price: 6.26770 USDT
    ✅ Futures Price: 6.30900 USDT
    ✅ Spread: 0.6589%
    ✅ Net Profit After Fees: 0.4589%
    ⚡ Execute arbitrage on Bybit
------------------------------------------------------------
✅ Found 553 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...
🔍 Arbitrage Opportunity on Bitget: LUCE/USDT | 2025-04-02 00:26:28 WIB
    ✅ Spot Price: 0.01071 USDT
    ✅ Futures Price: 0.01075 USDT
    ✅ Spread: 0.3735%
    ✅ Net Profit After Fees: 0.2135%
    ⚡ Execute arbitrage on Bitget
------------------